In [1]:
import boto3
import csv
import json
import os
import requests
import time
from io import BytesIO;
from os import listdir
from os.path import isfile, join

In [2]:
path = "/home/propietari/Documents/fitxers importants/WRI/Scraping_results/"

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]


In [3]:
onlyfiles

['ElSalvador_20210730.csv',
 'USFR_2_20210311.csv',
 'Mexico_20210423.csv',
 'USFR_20210311.csv',
 'USFR_1_20210311.csv',
 'USFR_20210702.csv',
 'LeyChile_20210702.csv',
 'India_20210310.csv',
 'Mexico_20210703.csv',
 'USFR_20210310.csv',
 'India_20210703.csv',
 'India_20210812_2011.csv',
 'Mexico_20210809.csv',
 'El_Salvador_no_duplicates.csv']

In [4]:
# path = 'C:/Users/jordi/Documents/claus/'
key_path = '/home/propietari/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = key_path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)


In [15]:
# Load the codes of all available documens

language = "spanish"
filename = f"{language}_final_documents.json"
file = path + filename

with open(file, 'r') as dict:
    codes = json.load(dict)


In [16]:
len(codes)

8833

### El Salvador

In [ ]:
# Loading the csv files from scraping to be compared
file_position = 0
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    ElSalvador_july = [row[3] for row in reader]

file_position = 13
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    El_Salvador_march = [row[7] for row in reader]

In [ ]:
print(f"There are {len(El_Salvador_march) - len(set(El_Salvador_march))} duplicates in the march version")
print(f"There are {len(ElSalvador_july) - len(set(ElSalvador_july))} duplicates in the august version")
print(f"There are {len(set(El_Salvador_march))} items in the March file and {len(set(ElSalvador_july))} in the August one")
print(f"There are {len(ElSalvador_july) - len(set(ElSalvador_july))} duplicates in the new version")
print(f"There are {len(set(El_Salvador_march) - set(ElSalvador_july))} files in the old version that are not in the new version")
print(f"There are {len(set(ElSalvador_july) - set(El_Salvador_march))} files in the new version that are not in the old version")
print(f"There are {len(set(El_Salvador_march) | set(ElSalvador_july))} unique ids in the union of the files")

In [ ]:

keys_dict = {key : 0 for key in list(set(El_Salvador_march) | set(ElSalvador_july))}

In [ ]:
path = "/home/propietari/Documents/GitHub/policy-data-analyzer/tasks/database_analysis/input/"
file_name = "ElSalvadorKeys.json"
file = path + file_name

with open(file, 'w') as fp:
    json.dump(keys_dict, fp)

In [43]:
# Going to merge the two files in one. The fields were not saved in the same order so it is necessary to rearrange them

file_position = 0
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    ElSalvador_july = list(reader)

file_position = 13
with open(path + onlyfiles[file_position], 'r') as f:
    reader = csv.reader(f)
    El_Salvador_march = list(reader)
    
final_list = []
ids = {}
for item in El_Salvador_march:
    if item[7] not in ids and item[7] in codes:
        final_item = []
        final_item.append(item[7])
        final_item.append('El Salvador')
        final_item.append('Diario Oficial')
        final_item.append(item[1])
        final_item.append(item[4])
        final_item.append(item[2])
        final_item.append(item[6])
        final_item.append(item[5])
        final_item.append(item[3])
        final_list.append(final_item)
        ids[item[7]] = 0
    
for item in ElSalvador_july:
    if item[3] not in ids and item[7] in codes:
        final_item = []
        final_item.append(item[3])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[7])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3]] = 0

In [28]:
len(final_list)

460

In [26]:
metadata_elements = {}

for item in El_Salvador_march:
    metadata_elements[item[7]] = 0
    
for item in ElSalvador_july:
    metadata_elements[item[3]] = 0
    
len(metadata_elements)

507

In [ ]:
# Checking the differences with bucket files

language = "spanish"
in_prefix = f"{language}_documents/HSSC/updated_sentences/"
ids_S3 = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".json" in obj.key and "ing" not in obj.key:
        name = obj.key.split("/")[-1].split("_")[0]
        ids_S3[name] = 0
        

In [ ]:
print(f"The number of scraped items that are not in sentences is {len(ids.keys() - ids_S3.keys())}")

In [ ]:
lst = list(ids.keys())
print(list(ids.keys() & ids_S3.keys()))

In [ ]:
# Here we go and fetch the missing files and put them in the raw_pdf_updated folder
prefix = f"{language}_documents/raw_pdf_updated/"


counter = 0
bucket = s3.Bucket(s3BucketName)
for item in final_list:
    if item[0] not in ids_S3:
        counter += 1
        key = prefix + item[0] + ".pdf"
        with requests.get(item[8], stream=True) as r:
            bucket.upload_fileobj(r.raw, key)
print(counter)

### Chile

In [44]:
file_position = 6
with open(path + onlyfiles[file_position], 'r', encoding="ISO-8859-1") as f:#, encoding = "Latin1"
    reader = csv.reader(f)
    chile = list(reader)
    
# final_list = []
# ids = {}
dates = {}
for item in chile:
    if item[3].split(".")[0] not in ids and item[3].split(".")[0] in codes:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
        dates[item[6]] = 0
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0


In [30]:
len(final_list)

860

In [31]:
metadata_elements = {}

for item in chile:
    metadata_elements[item[3].split(".")[0]] = 0

    
len(metadata_elements)

860

In [ ]:
print(f"The original file has {len(chile)} data points. Without duplicates there are:{len(ids)}")
print(f"The earliest date is {min(dates.keys())} and the nearest date is {max(dates.keys())}")

In [ ]:
# Checking the differences with bucket files

language = "spanish"
in_prefix = f"{language}_documents/HSSC/sentences/"
ids_S3 = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".json" in obj.key and "ing" not in obj.key:
        name = obj.key.split("/")[-1].split("_")[0]
        ids_S3[name] = 0

In [ ]:
print(f"The number of scraped items that are not in sentences is {len(ids.keys() - ids_S3.keys())}")

In [ ]:
# Here we go and fetch the missing files and put them in the raw_pdf_updated folder
prefix = f"{language}_documents/text_files/HSSC/new/"

counter = 0
for item in final_list:
    if item[0] not in ids_S3:
        counter += 1
        key = prefix + item[0] + ".txt"
        with requests.get(item[8], stream=True) as r:
            s3.Object(s3BucketName, key).put(Body=r.content)

print(counter)

### Mexico

In [ ]:
onlyfiles

In [7]:
# Data loading
file_position = 8
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:#, encoding="ISO-8859-1"
    reader = csv.reader(f)
    mexico_july = list(reader)
    
file_position = 12
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:#, encoding="ISO-8859-1"
    reader = csv.reader(f)
    mexico_august = list(reader)

id_july = [x[3].split(".")[0] for x in mexico_july]
id_august = [x[3].split(".")[0] for x in mexico_august]

code_july = [x[7] for x in mexico_july]
code_august = [x[7] for x in mexico_august]


In [8]:
mexico_july[0]

['PODER EJECUTIVO/SECRETARIA DE DESARROLLO AGRARIO, TERRITORIAL Y URBANO',
 'Mexico',
 'Diario Oficial de la Federacion',
 '60c51f636c3e97ba7251bce3e61411ae01905a59.txt',
 '',
 '',
 '02-06-2021',
 '5620125',
 'Federal',
 '',
 'Decreto por el que se aprueba el Programa Nacional de Ordenamiento Territorial y Desarrollo Urbano 2021-2024.',
 'https://sidofqa.segob.gob.mx/dof/sidof/notas/24-06-2021']

In [9]:
mexico_august[1]

['PODER EJECUTIVO/SECRETARIA DE ENERGIA',
 'Mexico',
 'Diario Oficial de la Federacion',
 'cb1b92eab32cd9946da8c5177858ecf34fc729aa.txt',
 'https://www.dof.gob.mx/nota_detalle.php?codigo=5173785&fecha=05/01/2011&print=true',
 '',
 '05-01-2011',
 '5173785',
 'Federal',
 '',
 'Proyecto de Modificación a la Norma Oficial Mexicana NOM-008- NUCL-2003, Control de la contaminación radiactiva',
 'https://sidofqa.segob.gob.mx/dof/sidof/notas/05-01-2011']

In [ ]:
print(f"July {len(id_july)} whithout dups {len(set(id_july))} - August {len(id_august)} whithout dups {len(set(id_august))}")
print(f"There are {len(set(id_july) - set(id_august))} files in the july dataset which are not in the august one")
print(f"There are {len(set(code_july) - set(code_august))} files by code in the july dataset which are not in the august one")

In [ ]:
# Build the conversion dictionary from old codes to new ones

july_conv = dict((key[7], key[3].split(".")[0]) for key in mexico_july)
august_conv = dict((key[7], key[3].split(".")[0]) for key in mexico_august)

conversor = {}
for code, ID in july_conv.items():
    conversor[ID] = august_conv[code]

In [ ]:
july_conv["c44f26663cde7e43afb625a4678de0b5f443a3ce"]

In [ ]:
august_conv["c44f26663cde7e43afb625a4678de0b5f443a3ce"]

In [ ]:
conversor

In [45]:

file_position = 8
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    Mexico_july = list(reader)

file_position = 12
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    Mexico_august = list(reader)
    
# final_list = []
# ids = {}
for item in Mexico_july:
    if item[3].split(".")[0] not in ids and item[3].split(".")[0] in codes:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0
    
for item in Mexico_august:
    if item[3].split(".")[0] not in ids and item[3].split(".")[0] in codes:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0

In [ ]:
print(f"The original file has {len(mexico)} data points. Without duplicates there are:{len(ids)}")
print(f"The earliest date is {min(dates.keys())} and the nearest date is {max(dates.keys())}")

In [46]:
len(final_list)

6419

In [37]:
metadata_elements = {}

# for item in Mexico_july:
#     metadata_elements[item[3].split(".")[0]] = 0
    
for item in Mexico_august:
    metadata_elements[item[3].split(".")[0]] = 0
    
len(metadata_elements)

5099

In [47]:
path = "/home/propietari/Documents/fitxers importants/WRI/Scraping_results/Final_DB_july_2021/"
file = path + "spanish_metadata.csv"
with open(file, 'w', newline='') as f:
     wr = csv.writer(f)
     wr.writerows(final_list)

In [ ]:
path = "/home/propietari/Documents/fitxers importants/WRI/Scraping_results/March2021/"

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]


In [ ]:
onlyfiles

In [ ]:
file = path + "Scraped_Documents_Chile.csv"
with open(file, 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    Chile = [row[13] for row in reader]

In [ ]:
Chile[1]

In [ ]:
print(f"There are {len(Chile)} entries and {len(set(Chile))} entries")

### India

In [ ]:
# Loading the csv files from scraping to be compared
file_position = 7
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    india_march = [row[3] for row in reader]

file_position = 11
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    india_august = [row[7] for row in reader]

In [ ]:
print(f"There are {len(india_march) - len(set(india_march))} duplicates in the march version")
print(f"There are {len(india_august) - len(set(india_august))} duplicates in the august version")
print(f"There are {len(set(india_march))} items in the March file and {len(set(india_august))} in the August one")
print(f"There are {len(set(india_march) - set(india_august))} files in the old version that are not in the new version")
print(f"There are {len(set(india_august) - set(india_march))} files in the new version that are not in the old version")
print(f"There are {len(set(india_march) | set(india_august))} unique ids in the union of the files")

In [5]:
file_position = 7
with open(path + onlyfiles[file_position], 'r', encoding="ISO-8859-1") as f:#, encoding = "Latin1"
    reader = csv.reader(f)
    india_march = list(reader)

file_position = 11
with open(path + onlyfiles[file_position], 'r', encoding="ISO-8859-1") as f:
    reader = csv.reader(f)
    india_august = list(reader)

In [43]:
india_march[5]

['',
 'India',
 'India Code',
 '43fd1c3c2b65e8a586a34995feb553be7de5ed8d.pdf',
 'https://upload.indiacode.nic.in/showfile?actid=AC_CEN_23_31_00002_200153_1517807322692&type=rule&filename=rules_2003_sw.pdf',
 'Rule',
 '12-09-2003',
 '',
 'Federal',
 '',
 'Protection of Plant Vhrieties and Farmersâ\x80\x99 Rights Rules, 2003.',
 'https://www.indiacode.nic.in/handle/123456789/4934?view_type=search&sam_handle=123456789/2513']

In [ ]:
india_august[5]

In [6]:
# Going to merge the two files in one. The fields were not saved in the same order so it is necessary to rearrange them

file_position = 7
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    india_march = list(reader)

file_position = 11
with open(path + onlyfiles[file_position], 'r') as f:
    reader = csv.reader(f)
    india_august = list(reader)
    
final_list = []
ids = {}
for item in india_march:
    if item[7] not in ids:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0
    
for item in india_august:
    if item[3] not in ids:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0

In [7]:
# Checking the differences with bucket files

language = "english"
in_prefix = f"{language}_documents/HSSC/sentences/"
ids_S3 = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".json" in obj.key and "ing" not in obj.key:
        name = obj.key.split("/")[-1].split("_")[0]
        ids_S3[name] = 0

In [35]:
language = "english"
in_prefix = f"{language}_documents/raw_pdf/"
ids_raw = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".pdf" in obj.key :
        name = obj.key.split("/")[-1].split("_")[0].split(".")[0]
        ids_raw[name] = 0

In [8]:
print(f"The number of scraped items that are not in sentences is {len(ids.keys() - ids_S3.keys())}")

The number of scraped items that are not in sentences is 1106


In [42]:
len(ids_raw)
final_list[1]

['876b66a43ea68e055193465940377107b9cd4fcc',
 'India',
 'India Code',
 ' Ministry of Agriculture and Farmers Welfare/ Department of \xa0Agriculture, Cooperation and Farmers Welfare',
 'Act',
 ' 2020-09-24',
 'The Farmers (Empowerment and Protection) Agreement on Price Assurance and Farm Services Act, 2020.',
 'An Act to provide for a national framework on farming agreements that protects and empowers farmers to engage with agri-business firms, processors, wholesalers, exporters or large retailers for farm services and sale of future farming produce at a mutually agreed remunerative price framework in a fair and transparent manner and for matters connected therewith or incidental thereto.',
 'https://www.indiacode.nic.in/bitstream/123456789/15511/1/A2020_20.pdf#search=Miner OR Ore OR Pit OR Bog OR Buffer OR Corridor OR (Country planning) OR Cropland OR (Degraded land) OR Desert OR Floodplain OR Forestland OR Freshwater OR Grassland OR (Land use) OR Landowner OR Mangrove OR Marsh OR Mead

In [37]:
missing = list(missing_Dict.keys() - ids_raw.keys())
remissing_Dict = {key: None for key in missing}

len(remissing_Dict)

462

In [46]:

# Here we go and fetch the missing files and put them in the raw_pdf_updated folder
prefix = f"{language}_documents/raw_pdf/"


counter_total = 0
counter_success = 0
missing = {}
bucket = s3.Bucket(s3BucketName)
for item in final_list:
    if item[0] in remissing_Dict:
        counter_total += 1
        key = prefix + item[0] + ".pdf"
        try:
            with requests.get(item[8], stream=True, verify=False) as r:
                bucket.upload_fileobj(r.raw, key)
            counter_success += 1
        except Exception as e:
            print(f"{e}\n{item[8]}\n")
            missing[item[0]] = 0 
        counter = str(counter_success) + " out of " + str(counter_total)
        print(counter, end = "\r")
        time.sleep(1)
            


/home/propietari/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.indiacode.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


/home/propietari/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.indiacode.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


/home/propietari/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.indiacode.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


/home/propietari/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.indiacode.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


/home/propietari/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.indiacode.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


...

/home/propietari/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.indiacode.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


/home/propietari/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.indiacode.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


/home/propietari/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.indiacode.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


/home/propietari/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.indiacode.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


/home/propietari/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'upload.indiacode.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


### USA

In [ ]:
# Going to merge the two files in one. The fields were not saved in the same order so it is necessary to rearrange them

file_position = 7
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    india_march = list(reader)

file_position = 11
with open(path + onlyfiles[file_position], 'r') as f:
    reader = csv.reader(f)
    india_august = list(reader)
    
final_list = []
ids = {}
for item in india_march:
    if item[7] not in ids:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0
    
for item in india_august:
    if item[3] not in ids:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0

In [30]:
missing

['...',
 ...]